<a href="https://colab.research.google.com/github/thiviyan98/ThiviyanNanthakumarCE888/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np

# random cells =99 as the jokes are not rated
df= pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv', header= None)

print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


##Helper Function

In [43]:
arr = df.values
print(arr.shape)

(24983, 101)


In [44]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1835357 (1835357,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [46]:
new_arr, idx = replace(arr, 0.1)

print(new_arr)
print(idx) #validation set

print(arr[idx[0][0], idx[1][0]])

print(new_arr[idx[0][0], idx[1][0]])

[[ 74.    -7.82   8.79 ...  99.    99.    99.  ]
 [100.     4.08  99.   ...   0.34  -4.32   1.07]
 [ 49.    99.    99.   ...  99.    99.    99.  ]
 ...
 [ 39.    99.    99.   ...  99.    99.    99.  ]
 [ 37.    99.    99.   ...  99.    99.    99.  ]
 [ 72.     2.43   2.67 ...  99.    99.    99.  ]]
(array([10343, 18940, 17600, ..., 22790,  1636, 13743]), array([52, 22, 32, ..., 51, 11, 60]))
7.28
99.0


##Latent Factor Modelling to infer Hidden Ratings of the users

In [0]:
n_latent_factors=5

user_ratings=arr

# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [63]:
latent_item_features

array([[0.62061279, 0.75298356, 0.70326791, 0.49374238, 0.99761423],
       [0.41230202, 0.20446941, 0.72844863, 0.35014588, 0.68777982],
       [0.45720823, 0.8432533 , 0.44977146, 0.31310766, 0.93023012],
       [0.00972908, 0.5996606 , 0.30465358, 0.6853633 , 0.56748746],
       [0.04413899, 0.35557657, 0.17385063, 0.87980118, 0.02798794],
       [0.24045911, 0.20764127, 0.41695158, 0.00735108, 0.62799764],
       [0.0326643 , 0.43206212, 0.6951509 , 0.90767309, 0.55748547],
       [0.48655825, 0.25732072, 0.80927818, 0.6709815 , 0.92202022],
       [0.62702023, 0.78971142, 0.23195712, 0.24345471, 0.87063273],
       [0.21143188, 0.09579828, 0.93418062, 0.1376297 , 0.70431716],
       [0.55348785, 0.61252862, 0.34900765, 0.31663752, 0.06434165],
       [0.05925061, 0.20116877, 0.4201659 , 0.88909519, 0.77189914],
       [0.87600722, 0.66351626, 0.97415118, 0.23122218, 0.43371947],
       [0.3364737 , 0.86922734, 0.87578086, 0.11963541, 0.51146348],
       [0.14118778, 0.63056745, 0.

In [64]:
latent_user_preferences

array([[0.86280176, 0.32338368, 0.8529395 , 0.0868466 , 0.3869725 ],
       [0.64578692, 0.48826018, 0.15565562, 0.1462454 , 0.34946404],
       [0.45072874, 0.43293574, 0.48888282, 0.86745475, 0.55191893],
       ...,
       [0.27590497, 0.62565027, 0.82901585, 0.74950984, 0.65526915],
       [0.85783496, 0.33111735, 0.47422199, 0.72001882, 0.47837717],
       [0.11002355, 0.51909169, 0.88641066, 0.91246758, 0.70186002]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating!=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [66]:
sgd()  # Note how the MSE decreases with the number of iterations

44.45123139813688
24.52230507100866
20.33469966275896
18.008656782244874
17.764682490487523


In [67]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[76.40960927, -3.92355721, -3.34703422, ..., -3.11132564,
        -5.21691397, -2.72526639],
       [95.59765272,  3.78821993,  2.59947252, ...,  2.28691532,
         2.27763338,  2.73349357],
       [49.18655772,  6.8244959 ,  6.88627138, ...,  7.56697259,
         7.40059282,  8.20857642],
       ...,
       [39.38899387,  0.74412662, -0.4437974 , ..., -0.86151815,
        -0.40678031, -0.8967705 ],
       [37.97507621, -0.84349164, -0.95564455, ..., -0.36959259,
        -1.01961784, -0.20849695],
       [73.77656052,  4.00683374,  2.99764661, ...,  2.97270589,
         2.99526457,  3.33677266]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [69]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 76.40960927031135)","(-7.82, -3.9235572129103917)","(8.79, -3.347034221512326)","(-9.66, -5.396669958254993)","(-8.16, -7.572088546949723)","(-7.52, -2.4071785514057615)","(-8.5, -2.6117241884293483)","(-9.85, -3.522410498449565)","(4.17, -3.1555450278387602)","(-8.98, -7.200075733005477)","(-4.76, -3.448342241413562)","(-8.5, -2.887489398672834)","(-6.75, -2.43286665202663)","(-7.18, -4.003695025305375)","(8.45, -2.2539016190190764)","(-7.18, -3.818701770094723)","(-7.52, -6.4739791882414695)","(-7.43, -3.1065647457955303)","(-9.81, -4.0203777122121584)","(-9.85, -3.4639126342667197)","(-9.85, -4.714242102549551)","(-9.37, -0.8776366483045787)","(1.5, -3.708607677731727)","(-4.37, -5.175844387546659)","(-9.81, -8.065270740822466)","(-8.5, -4.038089070928206)","(1.12, -2.0508421250062425)","(7.82, 1.1166499844174236)","(2.86, -2.4056097643199608)","(9.13, 1.6872420691298609)","(-7.43, -6.524335473611714)","(2.14, -0.22912823082291273)","(-4.08, 0.8972867333330112)","(-9.08, -7.304375191394359)","(7.82, -2.9251240500237254)","(5.05, 0.6649578965072781)","(4.95, 1.0769644511048573)","(-9.17, -6.8856045946860345)","(-8.4, -3.3047750446763535)","(-8.4, -2.770271936209071)",...,"(8.59, -0.6679031731167812)","(3.59, 0.1545549138777142)","(-6.84, -4.376653920084885)","(-9.03, -6.439110599111646)","(2.82, -0.19207708594633582)","(-1.36, -0.38263494395034153)","(-9.08, -7.03553065545323)","(8.3, -0.24510388019969803)","(5.68, -0.0850834313079723)","(-4.81, -4.5077162119779235)","(99.0, -5.564523863061898)","(99.0, -1.2447216091751023)","(99.0, -4.614820332407937)","(99.0, -6.822619161575739)","(99.0, -5.689667741699046)","(99.0, -2.291238086369809)","(99.0, -4.711464647746041)","(-9.42, -3.73840782980072)","(99.0, -5.347487748142723)","(99.0, -3.7115722311001527)","(99.0, -2.3980975985885724)","(-7.72, -4.194161685831164)","(99.0, -2.4702710435622293)","(99.0, -4.529908718716485)","(99.0, -4.007332943212668)","(99.0, -5.112224568852479)","(99.0, -2.489925029022472)","(99.0, -2.100940794973284)","(99.0, -0.22072885407106138)","(99.0, -4.335385152203975)","(2.82, -2.633877036087243)","(99.0, -3.7576114400418055)","(99.0, -1.6040057337383078)","(99.0, -4.497324839468421)","(99.0, -3.8582410046502194)","(99.0, -3.463740801018287)","(-5.63, -2.9694963026122965)","(99.0, -3.1113256388805985)","(99.0, -5.216913965703078)","(99.0, -2.725266391341411)"
1,"(100.0, 95.59765271841682)","(4.08, 3.7882199322388144)","(-0.29, 2.5994725199818256)","(6.36, 3.252926992334046)","(4.37, 1.4640467923901925)","(-2.38, 1.5525107903554247)","(-9.66, 3.342321096014008)","(-0.73, 0.9104917696102528)","(-5.34, 0.47194113849416763)","(8.88, 2.84230271943896)","(9.22, 4.1979277121678225)","(6.75, 3.9863474436816917)","(8.64, 3.3519378602875625)","(4.42, -1.126562050486532)","(7.43, 3.059961293816348)","(4.56, -0.920920277610044)","(-0.97, -1.7607351489702971)","(4.66, -0.6705091034702241)","(-0.68, 0.9049340976519074)","(3.3, 1.4814315608219615)","(-1.21, 0.2224555253194048)","(0.87, 3.7888353998426405)","(8.64, 3.5121322955866576)","(8.35, 2.9952754545749447)","(9.17, 1.7686875869868253)","(0.05, 2.6915464573571066)","(7.57, 2.8742914877171786)","(4.71, 4.363826938009091)","(0.87, 3.3030793633911815)","(-0.39, 4.03022301226058)","(6.99, 2.323466679632281)","(6.5, 3.4820234599917343)","(-0.92, 4.442129008471545)","(7.14, 1.4469435296691924)","(9.03, 2.942852531060633)","(-1.8, 4.435475614110307)","(0.73, 4.708409931658523)","(7.09, 1.486359303694802)","(3.4, 3.8884537526430254)","(-0.87, 2.8400062372221826)",...,"(-6.7, 3.8215569472508095)","(-3.35, 4.572920472667652)","(-9.03, 2.3128458647465595)","(4.47, 2.4004608694672904)","(4.08, 3.7271465286368586)","(-3.83, 4.0820501637900355)","(8.74, 2.5535800188418283)","(1.12, 4

##valid set

In [0]:
n_latent_factors=5

user_ratings=new_arr

# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating!=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [72]:
sgd()

45.81714597232098
24.495600223368964
21.812859323939602
18.09686422977399
17.786854421167423


In [73]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[76.94005212, -2.94561255, -3.44255855, ..., -2.91496868,
        -4.77158038, -3.17413902],
       [94.83871681,  2.61379355,  2.39936368, ...,  2.52428582,
         1.10052024,  3.23352708],
       [48.77405572,  5.9135078 ,  7.17657355, ...,  7.86793164,
         6.06280058,  8.49344846],
       ...,
       [39.72395664, -0.59705699, -1.18268985, ..., -1.28104652,
        -1.55618059, -1.01613034],
       [37.57139076, -0.48515849, -0.29867055, ...,  0.45881584,
        -1.0676126 ,  0.51710274],
       [72.67125347,  2.6484836 ,  2.62517564, ...,  3.56140119,
         2.15843647,  4.06058654]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [75]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 76.9400521230569)","(-7.82, -2.9456125454954916)","(8.79, -3.4425585460417056)","(-9.66, -4.137588886992969)","(-8.16, -6.167935791391625)","(-7.52, -2.507914897470397)","(99.0, -3.3581352618973397)","(-9.85, -3.015454313502595)","(4.17, -2.4266451620827048)","(-8.98, -6.154569572691248)","(-4.76, -2.967033198038608)","(-8.5, -2.2120223520258744)","(-6.75, -3.20189944479773)","(-7.18, -3.9419726114563427)","(8.45, -3.235979733444581)","(-7.18, -3.788203617199467)","(-7.52, -6.08545855942226)","(-7.43, -3.1877387449227403)","(-9.81, -3.846118334317243)","(-9.85, -3.535858227453599)","(-9.85, -4.932952107689312)","(-9.37, -1.8072485093583741)","(1.5, -3.8632655322759195)","(-4.37, -4.9267553800972586)","(-9.81, -6.475048751657094)","(-8.5, -4.321340303180858)","(1.12, -3.300417523897673)","(99.0, 0.02282348239246124)","(2.86, -3.3787360540971223)","(99.0, 0.2384537090484577)","(-7.43, -6.06567875932661)","(2.14, -1.5743890743373796)","(-4.08, -0.4472853519455423)","(-9.08, -5.984626513533182)","(7.82, -3.4881368830824373)","(5.05, -0.4637197129817041)","(4.95, 0.26952868840996197)","(-9.17, -5.696011384842879)","(-8.4, -3.637793811476831)","(-8.4, -3.3411910344829634)",...,"(8.59, -2.004011301810778)","(3.59, -1.2494177769556776)","(-6.84, -4.656058874527709)","(-9.03, -5.218056920346684)","(2.82, -1.4721530728484256)","(-1.36, -1.6862779050741252)","(-9.08, -6.211815638866935)","(8.3, -1.4501108587528668)","(5.68, -1.48218963035969)","(-4.81, -4.471571645764772)","(99.0, -4.825337947939476)","(99.0, -1.387275413226809)","(99.0, -3.743025000508478)","(99.0, -5.824503894921065)","(99.0, -4.631023072193923)","(99.0, -1.5470357648040025)","(99.0, -3.8108649161949715)","(-9.42, -2.935384438636297)","(99.0, -4.6322970779743935)","(99.0, -3.841918437311562)","(99.0, -2.3268151131594634)","(-7.72, -3.82169242936544)","(99.0, -1.8798043786981282)","(99.0, -4.064668743963569)","(99.0, -3.814605657225385)","(99.0, -4.27686111447296)","(99.0, -2.4641368999336497)","(99.0, -2.24472193069822)","(99.0, -0.7729328792533224)","(99.0, -4.2776265045796755)","(2.82, -2.502843952925656)","(99.0, -4.044328314772577)","(99.0, -1.9513171624317491)","(99.0, -3.800282691576512)","(99.0, -3.674290197278715)","(99.0, -3.40387563068517)","(-5.63, -2.7366436124782765)","(99.0, -2.914968678879668)","(99.0, -4.771580383686671)","(99.0, -3.1741390213541916)"
1,"(100.0, 94.8387168136915)","(4.08, 2.6137935496092237)","(99.0, 2.3993636799234195)","(6.36, 2.2641983927038316)","(4.37, 0.3491564453393824)","(-2.38, 1.8183951906902351)","(-9.66, 3.8870473579950535)","(-0.73, 0.48879811340807866)","(-5.34, -0.13739481837910772)","(8.88, 1.280522821084447)","(9.22, 2.905621407873131)","(6.75, 3.9481809424018475)","(8.64, 4.16122162937895)","(4.42, -1.2224498420088197)","(7.43, 4.173885282669893)","(4.56, -1.3833107957738946)","(-0.97, -3.087678704455277)","(99.0, -0.2581348075036763)","(99.0, 0.1268862826565571)","(3.3, 1.4732382908696897)","(-1.21, 0.14560048070314346)","(0.87, 4.883815309871489)","(8.64, 3.2681826574920882)","(8.35, 1.9827402442130784)","(9.17, -0.3070309999540917)","(0.05, 2.649823389185631)","(7.57, 3.8498233034835665)","(4.71, 5.464536665639612)","(0.87, 3.7670720544020972)","(-0.39, 5.49122223339111)","(99.0, 1.4014375199338296)","(6.5, 4.799384126062849)","(-0.92, 5.689653410098613)","(99.0, 0.2806668085750664)","(9.03, 3.4513935153507074)","(-1.8, 5.628022395820325)","(0.73, 5.554095860340108)","(99.0, -0.276223160477016)","(3.4, 3.2356888155703816)","(-0.87, 3.5248212924769557)",...,"(-6.7, 4.986855057290966)","(-3.35, 5.493276071733896)","(-9.03, 1.9001139139959502)","(4.47, 0.7060491706451064)","(4.08, 5.06093293034128)","(-3.83, 5.120528320825855)","(8.74, 0.9224125848001792)","(1.12, 4.91928103978

In [0]:
## the performance of the algorithm on the validation dataset was worse than on the orignal dataset. 


